In [166]:
import numpy as np
from joblib import load
from google.colab import drive

In [167]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [168]:
trained_model = load("/content/drive/MyDrive/ST2IASE/transpilers/linear.joblib")

In [169]:
slope = trained_model.coef_
intercept = trained_model.intercept_

In [170]:
nb_features = slope.size
print("Il y a",nb_features,"features.")

Il y a 3 features.


#Déclaration des I/O du module

In [171]:
code = "module linear_regression("

In [172]:
for i in range (nb_features):
  code += f"input [15:0] X{i}, "
code += "output [31:0] y);\n\n"

#Création des paramètres du modèle

In [173]:
code += f"reg [15:0] INTERCEPT = 16'd{intercept};\n"

In [174]:
for i in range (nb_features):
  code+= f"reg [15:0] SLOPE{i} = 16'd{slope[i]};\n"
code+="\n"

#Création des variables utiles (résultats de produit, sommes, retenues)

In [175]:
for i in range (nb_features):
  code+= f"wire [31:0] prod_res{i};\n"
code+="\n"

In [176]:
for i in range (nb_features):
  code+= f"wire c_int_{i},c_out_{i};\n"
code+="\n"

In [177]:
code += "wire [31:0] "
for i in range (nb_features):
  code+= f"sum{i},"
code+=";\n\n"

#Création des modules de produits

In [178]:
for i in range (nb_features):
  code+= f"multiplier16 prod{i}(X{i},SLOPE{i},prod_res{i});\n"

#Création des modules de sommes

In [179]:
code += f"""
adder16 adder0(prod_res0[15:0],INTERCEPT,16'd0,sum0[15:0],c_int_0);
adder16 adder1(prod_res0[31:16],16'd0,c_int_0,sum0[31:16],c_out_0);
"""

In [180]:
j = 0
for i in range(1,nb_features):
  code += f"""
adder16 adder{i+1+j}(sum{i-1}[15:0],prod_res{i}[15:0],16'd0,sum{i}[15:0],c_int_{i});
adder16 adder{i+2+j}(sum{i-1}[31:16],prod_res{i}[31:16],c_int_{i},sum{i}[31:16],c_out_{i});
"""
  j += 1

In [181]:
code += f"""
assign price = sum{nb_features-1};

end module;
"""

In [182]:
with open('linear_regression.v', 'w') as f:
    f.write(code)

In [183]:
print(code)

module linear_regression(input [15:0] X0, input [15:0] X1, input [15:0] X2, output [31:0] y);

reg [15:0] INTERCEPT = 16'd-8152.937710156519;
reg [15:0] SLOPE0 = 16'd717.2583697096838;
reg [15:0] SLOPE1 = 16'd36824.195974256305;
reg [15:0] SLOPE2 = 16'd101571.84002157034;

wire [31:0] prod_res0;
wire [31:0] prod_res1;
wire [31:0] prod_res2;

wire c_int_0,c_out_0;
wire c_int_1,c_out_1;
wire c_int_2,c_out_2;

wire [31:0] sum0,sum1,sum2,;

multiplier16 prod0(X0,SLOPE0,prod_res0);
multiplier16 prod1(X1,SLOPE1,prod_res1);
multiplier16 prod2(X2,SLOPE2,prod_res2);

adder16 adder0(prod_res0[15:0],INTERCEPT,16'd0,sum0[15:0],c_int_0);
adder16 adder1(prod_res0[31:16],16'd0,c_int_0,sum0[31:16],c_out_0);

adder16 adder2(sum0[15:0],prod_res1[15:0],16'd0,sum1[15:0],c_int_1);
adder16 adder3(sum0[31:16],prod_res1[31:16],c_int_1,sum1[31:16],c_out_1);

adder16 adder4(sum1[15:0],prod_res2[15:0],16'd0,sum2[15:0],c_int_2);
adder16 adder5(sum1[31:16],prod_res2[31:16],c_int_2,sum2[31:16],c_out_2);

assign pri